In [13]:
import numpy as np
import torch
import torch.nn as nn

import neurogym as ngym

from tasks import AverageDirectionTest
from tasks import PerceptualDecisionMaking

# Environment
env = PerceptualDecisionMaking()
_ = env.reset()
ob_size = env.observation_space.shape[0]
act_size = env.action_space.n
num_neurons = ob_size

In [14]:
#Initialize variables
alpha = 0.1
training_episodes = 200
phi = np.zeros((1, ob_size))          # (1, obs_size) denotes inputs from environment, zeros are placeholder for what the actual input values are
w = np.random.randn(num_neurons, ob_size)  # (obs_size, 1) weights used to learn importance of each input
#create sparse array B
B = np.zeros((num_neurons, ob_size)) #(num_neurons, num_inputs) represents weighted combination of what input values each neuron sees
num_elements = num_neurons * ob_size
sparsity = 0.2
indices = np.random.choice(num_elements, round(num_elements*sparsity), replace=False)
B.flat[indices] = np.random.randn(round(num_elements*sparsity))

In [16]:
print(B.shape)
print(w.shape)
print(phi.shape)
phi, gt = env.ob, env.gt
print(phi.shape)


(3, 3)
(3, 3)
(1, 3)
(22, 3)


In [ ]:
#Train loop -- need to figure out how to get scalar V for each neuron
for i in training_episodes: 
  trial = env.new_trial()
  phi, gt = env.ob, env.gt
  V = w @ phi.T                  # (num_neurons, num_objects) list of values for each neuron
  D =  B @ V                      # (num_neurons, 1) list of decisions between left (0) and right (1)
  action = np.clip(round(D), 0, 1)
  phi, reward, done, misc_dict = env.step(action)
  # r = env.get_reward_vals         # (num_neurons, 1) correct decisions -- get_reward_vals needs to return the correct decision given the environment state
  delta = abs(D - reward)              # (num_neurons, 1) error between decision and correct decision
  w = w + alpha*(B.T*delta)       #w update